In [1]:
%pip install emoji
%pip install pandas
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

data = pd.read_excel('scrapped_data.xlsx')
data

C:\Users\halim\AppData\Local\Temp\ipykernel_12372\3525272284.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,score,at,content
0,5,2023-10-13 15:11:58,Aplikasinya bagus semua nya bagus tapi tolong ...
1,5,2023-10-13 12:43:50,"Lewat aplikasi ruang guru,saya bisa tau pembel..."
2,5,2023-10-13 11:07:57,Seru isinya bisa ngerjain tugas dr aplikasi at...
3,5,2023-10-12 20:23:34,Baik
4,3,2023-10-12 11:21:43,Jgn di dwnld! Nanti bayar
...,...,...,...
13939,5,2021-12-22 06:16:54,Keren
13940,2,2021-12-22 05:54:42,gw ngga ngerti cara mainnya
13941,5,2021-12-22 05:35:53,🤝🏻
13942,1,2021-12-22 05:06:25,Paket 3 bulan Explorers hanya masuk 3x.mengapa...


In [3]:
import emoji
import re

def split_text_and_symbols(text):
    if isinstance(text, str):
        words = re.findall(r'\b\w+\b', text)
        emoticon_set = set(char for char in text if char in emoji.EMOJI_DATA)
        emoticon_list = [emoji.emojize(emoticon) for emoticon in emoticon_set]
        sentence = ' '.join(words)

        return sentence, emoticon_list
    else:
        return '', [] 

In [4]:
dataset = pd.DataFrame()

dataset[['text', 'emoticon']] = data['content'].apply(split_text_and_symbols).apply(pd.Series)
dataset['rating'] = data['score']
dataset.head()

,text,emoticon,rating
0,Aplikasinya bagus semua nya bagus tapi tolong ...,"[🏻, 👍]",5
1,Lewat aplikasi ruang guru saya bisa tau pembel...,[],5
2,Seru isinya bisa ngerjain tugas dr aplikasi at...,[],5
3,Baik,[],5
4,Jgn di dwnld Nanti bayar,[],3


In [5]:
import pandas as pd

def load_lexicon_from_excel(file_path):
    try:
        df = pd.read_excel(file_path)
        lexicon = {row['emoticon'].strip().replace("'", ""): int(row['polarity'])
                   for _, row in df.iterrows()
                   if isinstance(row['polarity'], int)}
        return lexicon
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None
    except Exception as e:
        print("Error:", e)
        return None

def calculate_sentiment(symbols, lexicon):
    sentiment_score = 0
    unprocessable_emoticons = []
    if symbols:
        sentiment_score = sum(lexicon.get(char, 0) for char in symbols)
        unprocessable_emoticons = [char for char in symbols if char not in lexicon]
    return sentiment_score, unprocessable_emoticons

nama_file_lexicon = 'dataset/emoticon.xlsx'

lexicon = load_lexicon_from_excel(nama_file_lexicon)

if lexicon is not None:
    sentiment_data = dataset['emoticon'].apply(lambda x: calculate_sentiment(x, lexicon))
    dataset['polarity_emotion'] = [sent[0] if sent[1] != [] else 0 for sent in sentiment_data]
    dataset['unprocessable_emoticons'] = [sent[1] for sent in sentiment_data]

    total_emoticons = len(dataset)
    emoticons_processed = total_emoticons - sum(1 for emo_list in dataset['unprocessable_emoticons'] if emo_list)
    success_percentage = (emoticons_processed / total_emoticons) * 100
    failure_percentage = 100 - success_percentage

    print(f"Total emoticons: {total_emoticons}")
    print(f"Emoticons processed: {emoticons_processed} ({success_percentage:.2f}%)")
    print(f"Emoticons failed to process: {total_emoticons - emoticons_processed} ({failure_percentage:.2f}%)")
else:
    print("Failed to load lexicon. Cannot continue computation.")

dataset.head()

Total emoticons: 13944
Emoticons processed: 12507 (89.69%)
Emoticons failed to process: 1437 (10.31%)


,text,emoticon,rating,polarity_emotion,unprocessable_emoticons
0,Aplikasinya bagus semua nya bagus tapi tolong ...,"[🏻, 👍]",5,1,[🏻]
1,Lewat aplikasi ruang guru saya bisa tau pembel...,[],5,0,[]
2,Seru isinya bisa ngerjain tugas dr aplikasi at...,[],5,0,[]
3,Baik,[],5,0,[]
4,Jgn di dwnld Nanti bayar,[],3,0,[]


In [6]:
unique_emoticons = set()

for value in dataset['unprocessable_emoticons']:
    if isinstance(value, list) and value:
        unique_emoticons.update(value)

filtered_df = pd.DataFrame(list(unique_emoticons), columns=['emoticons'])

#simpan data emoticon yang tidak bisa klasifikasi
filtered_df.to_excel('gagal_proses/unique_emoticons.xlsx', index=False)

#hapus kolom unprocessable_emoticons tidak butuh
dataset.drop('unprocessable_emoticons', axis=1, inplace=True)
#hasil akhir menjadi dataset
dataset.to_excel('hasil/dataset.xlsx', index=False)

dataset.head()

,text,emoticon,rating,polarity_emotion
0,Aplikasinya bagus semua nya bagus tapi tolong ...,"[🏻, 👍]",5,1
1,Lewat aplikasi ruang guru saya bisa tau pembel...,[],5,0
2,Seru isinya bisa ngerjain tugas dr aplikasi at...,[],5,0
3,Baik,[],5,0
4,Jgn di dwnld Nanti bayar,[],3,0
